Script adaptado para 1074 pontos artificiais

In [ ]:
# Install dependencies
!pip install -q amplpy

# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["gurobi", "cplex", "highs"], # modules to install
    g=globals()) # instantiate AMPL object and register magics

ampl.option["solver"] = "cplex"

In [ ]:
%%writefile modelo.mod
set escolas;
set vertices;
set regioes_sjc;

# Dados extraídos
param distancias{vertices, escolas};
param capacidade{escolas};
param demanda{vertices};
param escola_regiao_sjc{escolas, regioes_sjc};

# Parâmetros da instância
param custo_construir_sala;
#param num_medianas;
param num_escolas_candidatas;
param M;
param num_medianas;


# Variáveis de decisão
var x{escolas} binary;
var y{vertices,escolas} binary;
var s{escolas} integer >= 0;

#Função-objetivo
minimize Z: sum{i in vertices, j in escolas} (distancias[i,j] * y[i,j]) + sum{j in escolas}custo_construir_sala*s[j];

# Restrições
s.t. R1: sum{j in escolas} x[j] = num_medianas;
s.t. R3 {i in vertices}: sum{j in escolas} y[i,j] = 1;
s.t. R4 {j in escolas}: capacidade[j]*x[j] - 0.25*sum{i in vertices}demanda[i]*y[i,j] >= 0;
s.t. R5 {j in escolas}: s[j] <= M*x[j];
s.t. R6 {k in regioes_sjc}: sum{j in escolas}x[j]*escola_regiao_sjc[j,k] >= 1;

In [ ]:
import pandas as pd

In [ ]:
distancias_centroides = pd.read_csv("matriz_dist_centroide_escola_ARTIFICIAL.csv", decimal=',', index_col=0)

In [ ]:
capacidade_atual = pd.read_csv("capacidade-atual-escolas-ARTIFICIAL.csv", index_col=0)

In [ ]:
demanda = pd.read_csv("populacao_setores_sjc_demanda.csv", index_col=0)
demanda = demanda.sort_index()

In [ ]:
escolas_regiao = pd.read_csv("escolas-regiao-ARTIFICIAL.csv", index_col=0)
escolas_regiao = escolas_regiao.fillna(0)

In [ ]:
# Extrai valores únicos para identificação
id_escolas = capacidade_atual.index.values
id_vertices = demanda.index.values
regioes_sjc = escolas_regiao.unstack().index.levels[0].values

In [ ]:
%%ampl_eval
reset;
model modelo.mod;

In [ ]:
ampl.set["escolas"] = id_escolas
ampl.set["vertices"] = id_vertices
ampl.set["regioes_sjc"] = regioes_sjc

In [ ]:
distancias_centroides_ampl = distancias_centroides.transpose().unstack()
idx = distancias_centroides_ampl.index
distancias_centroides_ampl.index = distancias_centroides_ampl.index.set_levels(idx.levels[1].astype("int64"), level=1)

In [ ]:
distancias_centroides_ampl = distancias_centroides_ampl.sort_index()

In [ ]:
ampl.param["distancias"] = distancias_centroides_ampl

In [ ]:
distancias_centroides_ampl[354990405000010][0]

In [ ]:
%%ampl_eval
display distancias[354990405000010,0];

In [ ]:
teste_distancias = ampl.get_parameter("distancias").getValues().toPandas()
new_levels = [level.astype(int) for level in teste_distancias.index.levels]
teste_distancias.index.set_levels(new_levels, inplace=True)
teste_distancias = teste_distancias.unstack()
teste_distancias.columns = teste_distancias.columns.get_level_values(1)

In [ ]:
teste_distancias[0][354990405000010]

In [ ]:
for i in id_vertices:
  for j in id_escolas:
    if (teste_distancias[j][i] != distancias_centroides_ampl[i][j]):
      print("ERRO COM i=",i," j=",j)

In [ ]:
ampl.param["capacidade"] = capacidade_atual

In [ ]:
%%ampl_eval
display capacidade[30];

In [ ]:
ampl.param["demanda"] = demanda

In [ ]:
%%ampl_eval
display demanda[354990405000023];

In [ ]:
ampl.param["escola_regiao_sjc"] = escolas_regiao.transpose().unstack()

In [ ]:
%%ampl_eval
display regioes_sjc;

In [ ]:
%%ampl_eval
display escola_regiao_sjc[207,'Sul'];
display escola_regiao_sjc[207,'Zona Rural'];

In [ ]:
ampl.param["custo_construir_sala"] = 10000
ampl.param["num_medianas"] = 538
ampl.param["num_escolas_candidatas"] = 1074
ampl.param["M"] = 1000000

In [ ]:
%%ampl_eval
solve;

In [ ]:
%%ampl_eval
display x;

In [ ]:
%%ampl_eval
option display_width 790;
#option cplex_options 'timelimit=300'

In [ ]:
%%ampl_eval
display y;

In [ ]:
%%ampl_eval
display s;

In [ ]:
%%ampl_eval
display num_medianas;

In [ ]:
%%ampl_eval
display _solve_time;

In [ ]:
saida_ampl = ampl.get_variable("y").getValues().toPandas()

In [ ]:
new_levels = [level.astype(int) for level in saida_ampl.index.levels]
saida_ampl.index.set_levels(new_levels, inplace=True)

In [ ]:
saida_ampl = saida_ampl.unstack()
saida_ampl

In [ ]:
saida_ampl.columns = saida_ampl.columns.get_level_values(1)

In [ ]:
saida_ampl = saida_ampl.astype(int)

In [ ]:
saida_ampl

In [ ]:
saida_ampl.to_csv("saida_ampl_artificial.csv", index_label='ID', sep=";")